In [4]:
#import bibtexparser
import pandas as pd
import numpy as np
import os
import glob
import yaml
import json
import sqlite3

from functions import write_yaml
from functions import write_json
from functions import write_csv
from functions import read_yaml
from functions import api_ieee
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [2]:
#Reading yaml config file[]
config_query = read_yaml("../05_Config/config_query.yaml")
display("config_query: ", config_query)


../05_Config/config_query.yaml read successfully


'config_query: '

{'call_api_ieee': True,
 'max_api_pagination': 2,
 'api_ieee_field_search': 'article_title',
 'api_ieee_string_search': 'big+data',
 'use_query_sql_bib_csv': True,
 'use_query_sql_ieee': True,
 'query_bib_csv': "select * from bib_csv WHERE abstract LIKE '%big%' AND abstract LIKE '%data%'",
 'query_ieee': "select * from api_ieee WHERE  abstract LIKE '%big%' AND abstract LIKE '%data%'",
 'output_extensions': ['json', 'csv']}

In [3]:
if config_query['call_api_ieee'] == True:
    api_ieee(   field_search = config_query['api_ieee_field_search'], 
                string_search = config_query['api_ieee_string_search'], 
                max_pagination = config_query['max_api_pagination'])

../05_Config/config_api.yaml read successfully
config_api:  {'apikey': '3jk2p7za5jtd7a39gwknygvk', 'format': 'json', 'max_records': 1000, 'sort_order': 'asc', 'sort_field': 'article'}
URL:  http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey=3jk2p7za5jtd7a39gwknygvk&format=json&max_records=1000&start_record=1&sort_order=asc&sort_field=article_number&article_title=big+data

total_records: 11163
total_searched: 5785569
max_pagination:  2
Requesting: 1 - 200
Requesting: 201 - 400
(400, 30)

df_api_raw:  (400, 30)
df_api:  (400, 8)


/Users/amigosdadancamooca/Documents/Impacta/Python_for_Data_Engineers/02_Code/functions.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_api_raw.authors[i] = str(df_api_raw.authors[i]).split("'full_name': '", maxsplit = 1)[-1].split("'", maxsplit = 1)[0]
/Users/amigosdadancamooca/Documents/Impacta/Python_for_Data_Engineers/02_Code/functions.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_api_raw.authors[i] = np.nan


api_ieee gravada com sucesso no banco SQLite


# SQL QUERIES

In [13]:
#Reading yaml config file[]
config_query = read_yaml("../05_Config/config_query.yaml")

#Connecting to SQLite db
dbfile = '/Users/amigosdadancamooca/Documents/Impacta/Python_for_Data_Engineers/03_OutputFiles/doi.db'
db = sqlite3.connect(dbfile)

if config_query['use_query_sql_bib_csv'] == True:
    tabela ='bib_csv'
    df_query_bib_csv = pd.read_sql_query(config_query['query_bib_csv'], db)
    print('query_bib_csv: ',config_query['query_bib_csv'])
    print(f'\nquery_bib_csv: {df_query_bib_csv.shape} results', db)
    #display(df_query_bib_csv.head())
    

if config_query['use_query_sql_ieee'] == True:
    tabela ='api_ieee'
    df_query_ieee = pd.read_sql_query(config_query['query_ieee'], db)
    print('\nquery_ieee: ',config_query['query_ieee'])
    print(f'\nquery_ieee: {df_query_ieee.shape} results', db)
    #display(df_query_ieee.head())

db.commit()
db.close()

#########################################################################

#Exporting based on the yaml config file
#print("Output extensions options: ", configuration_file['output_extensions'])

if 'csv' in config_query['output_extensions']:
    if config_query['use_query_sql_bib_csv'] == True:
        write_csv(df_query_bib_csv, f'df_query_bib')
    if config_query['use_query_sql_ieee'] == True:
        write_csv(df_query_ieee, f'df_query_ieee')

if 'json' in config_query['output_extensions']:
    if config_query['use_query_sql_bib_csv'] == True:
        write_json(df_query_bib_csv, f'df_query_bib')
    if config_query['use_query_sql_ieee'] == True:
        write_json(df_query_ieee, f'df_query_ieee')

if 'yaml' in config_query['output_extensions']:
    if config_query['use_query_sql_bib_csv'] == True:
        write_yaml(df_query_bib_csv, f'df_query_bib')
    if config_query['use_query_sql_ieee'] == True:
        write_yaml(df_query_ieee, f'df_query_ieee')

if not config_query['output_extensions']:
    raise Exception("Please, select a valid output file extension [json, csv, yaml] in the config_query.yaml")

print('\n','#' * 50,'\n Process completed successfully','\n','#' * 50)

../05_Config/config_query.yaml read successfully
query_bib_csv:  select * from bib_csv WHERE abstract LIKE '%big data%' OR abstract LIKE '%quality%'

query_bib_csv: (308, 34) results <sqlite3.Connection object at 0x11ee3c030>

query_ieee:  select * from api_ieee WHERE  abstract LIKE '%big data%' OR abstract LIKE '%quality%'

query_ieee: (275, 8) results <sqlite3.Connection object at 0x11ee3c030>

df_query_bib.csv successfully written

df_query_ieee.csv successfully written

df_query_bib.json successfully written

df_query_ieee.json successfully written

 ################################################## 
 Process completed successfully 
 ##################################################
